# Visualize the tsdf 
Goal of the notebook
1. space estimated by NNR 
2. Whether it can be passed as input for If-net

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import open3d as o3d
import numpy as np
import ipyvolume as ipv

import sys 
sys.path.append("../")
sys.path.append("../../")


### Compare different TSDFVolume

In [14]:
datadir = "/media/srialien/Elements/AT-Datasets/DeepDeform/new_test/foxWDFS_Actions2_cam1/"
color = o3d.io.read_image(os.path.join(datadir,'color',"0000.png"))
depth = o3d.io.read_image(os.path.join(datadir,'depth',"0000.png"))

rgbd = o3d.geometry.RGBDImage.create_from_color_and_depth(
    color, depth, depth_trunc=4.0, convert_rgb_to_intensity=False)

camera = np.loadtxt(os.path.join(datadir,"intrinsics.txt"))
camera = o3d.camera.PinholeCameraIntrinsic(512,500,*camera[ [0,1,0,1],[0,1,2,2]])

vol_dim = 256
voxel_size = 4/256
o3d_volume = o3d.pipelines.integration.UniformTSDFVolume(vol_dim*voxel_size,
vol_dim,
0.4,
o3d.pipelines.integration.TSDFVolumeColorType.RGB8,np.array([-1.5199234,-0.7890313,0.]))

o3d_volume_scalable = o3d.pipelines.integration.ScalableTSDFVolume(voxel_length=4/512,sdf_trunc=0.04,
color_type=o3d.pipelines.integration.TSDFVolumeColorType.RGB8)

o3d_volume.integrate(
    rgbd,
    camera,
    np.eye(4))

o3d_volume_scalable.integrate(
    rgbd,
    camera,
    np.eye(4))

mesh_open3d = o3d_volume.extract_triangle_mesh()

mesh_open3d_scalable = o3d_volume_scalable.extract_triangle_mesh()
vertices = np.asarray(mesh_open3d_scalable.vertices)
c_red = (0. / 255., 0 / 255., 0 / 255.)
mesh_open3d_scalable.vertex_colors = o3d.utility.Vector3dVector(np.tile(np.array(c_red).reshape((1,3)),(vertices.shape[0],1)))

In [24]:
o3d.visualization.draw_geometries([mesh_open3d,mesh_open3d_scalable])

In [32]:
ar = np.asarray(o3d_volume.extract_volume_tsdf())
ar[ar<0].shape

(45469,)

In [22]:
mesh_open3d_scalable 

TriangleMesh with 11641 points and 20729 triangles.

### Plot tsdf output

In [3]:
tsdf_path = "/media/srialien/Elements/AT-Datasets/DeepDeform/new_test/foxWDFS_Actions2_cam1/results/tsdf/105.pkl"
import pickle 
with open(tsdf_path,"rb") as f: 
    data = pickle.load(f)
tsdf_vol,color_vol,weight_vol = data
# from tsdf import TSDFVolume
# truncated_region = np.ones_like(tsdf_vol)
# TSDFVolume.compute_truncated_region(tsdf_vol,truncated_region,1.2)
# ipv.quickvolshow(tsdf_vol,level=0,opacity=[1,1,1], level_width=0.1)

## Use open3d UniformTSDF 

In [33]:
inject_data = data[[0,2]].reshape(2,-1).T
print(inject_data[inject_data!=0][::100])

[1. 1. 1. ... 1. 1. 1.]


In [11]:
inject_data = data[[0,2]].reshape(2,-1).T
inject_data = o3d.utility.Vector2dVector()
vol_dim = 256
voxel_size = 4/256
volume = o3d.pipelines.integration.UniformTSDFVolume(vol_dim*voxel_size,
vol_dim,
0.4,
o3d.pipelines.integration.TSDFVolumeColorType.RGB8,np.array([-1.5199234,-0.7890313,0.]))
volume.inject_volume_tsdf(inject_data) 

In [12]:
mesh = volume.extract_triangle_mesh()
o3d.visualization.draw_geometries([mesh])

In [13]:
mesh

TriangleMesh with 0 points and 0 triangles.

In [ ]:
# from skimage import measure
# verts = measure.marching_cubes(tsdf, level=0)[0]
# ipv.scatter(verts[:,0],verts[:,1],verts[:,2])

verts, faces, norms, vals = measure.marching_cubes(tsdf, level=0) 
ipv.plot_trisurf(verts[:,0],verts[:,1],verts[:,2],triangles=faces)

### Input to if-net

In [ ]:
# if_net_input_datapath = "/media/srialien/Elements/AT-Datasets/if-net"

In [ ]:
# def load_xyz(filepath):
#     with open(filepath,'r') as f: 
#         data = f.read().split('\n')[1:]
#         data = [ [float(x) for x in line.split()] for line in data if len(line) > 0]
#     return np.array(data)

# # Check the data
# for file in os.listdir(if_net_input_datapath):
#     print(file)
#     if "npz" in file:
#         data = np.load(os.path.join(if_net_input_datapath,file),allow_pickle=True)
#         for k in data.keys():
#             print(k,data[k].shape)
#     if "xyz" in file: 
#         data = load_xyz(os.path.join(if_net_input_datapath,file))
#         print(data.shape)

# Load occupancy grid  
# data = np.load("/media/srialien/Elements/AT-Datasets/if-net/voxelized_point_cloud_128res_1000points_incomplete_pc-0056_animals.npz",allow_pickle=True)
# V = data["compressed_wnum_occ"].reshape([128]*3)     
# V = V>0
# print(V.min(),V.max())        
# ipv.quickvolshow(V,opacity=0.03, level_width=0.1)

In [ ]:
# Import modules 
from tsdf import TSDFVolume # Create main TSDF module where the 3D volume is stored
from vis import get_visualizer # Visualizer 
from frame_loader import RGBDVideoLoader
from embedded_deformation_graph import EDGraph # Create ED graph from mesh, depth image, tsdf 
from run_lepard import Lepard_runner # SceneFlow module 
from NonRigidICP.model.registration_fusion import Registration as PytorchRegistration
from lepard.inference import Lepard	
from warpfield import WarpField # Connects ED Graph and TSDF/Mesh/Whatever needs to be deformed  
from test_utils import Dict2Class

In [ ]:
# def test1(use_gpu=True): # Plot TSDF
use_gpu = True
fopt = Dict2Class({"source_frame":0,\
    "gpu":use_gpu,"visualizer":"ipyvolume",\
    "datadir": "/media/srialien/Elements/AT-Datasets/DeepDeform/new_test/foxWDFS_Actions2_cam2",\
    "skip_rate":15,
    "voxel_size": 0.01,
    "voxel_dim": None})
vis = get_visualizer(fopt)

frame_loader = RGBDVideoLoader(fopt.datadir)

source_frame_data = frame_loader.get_source_data(fopt.source_frame)

source_mask = source_frame_data["im"][-1] > 0
source_pcd = source_frame_data["im"][3:,source_mask].T
mask_indices = np.where(source_mask)
bbox = [np.min(mask_indices[1]),np.min(mask_indices[0]),np.max(mask_indices[1]),np.max(mask_indices[0]),] # x_min, y_min, x_max,y_max


# intrinsics
cam_intr = np.eye(3)
cam_intr[0, 0] = source_frame_data["intrinsics"][0]
cam_intr[1, 1] = source_frame_data["intrinsics"][1]
cam_intr[0, 2] = source_frame_data["intrinsics"][2]
cam_intr[1, 2] = source_frame_data["intrinsics"][3]

max_depth = source_frame_data["im"][-1].max()

# Create a new tsdf volume
tsdf = TSDFVolume(bbox,max_depth+1, source_frame_data["intrinsics"], fopt,vis)
vis.tsdf = tsdf
tsdf.integrate(source_frame_data)

ipv.quickvolshow(tsdf.get_volume()[0],level=[0,0,0],data_min=-1, data_max=1)

# abc = vis.plot_tsdf_volume()
# test1()

In [ ]:
asdsa = tsdf.get_volume()[0]
asdsa.mean(),asdsa.min(),asdsa.max()

In [ ]:
source_pcd.max(axis=0),source_pcd.min(axis=0)

In [ ]:
bbox